# Исследование надежности заемщиков

## Открытие таблицы и изучение общей информацию о данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv') # чтение файла

In [3]:
data.head(20) # первые 20 строчек датафрейма data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Необходимо обработать значения в этом столбце: заменить все отрицательные значения положительными с помощью метода `abs()`.

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median') # медианное значение трудового стажа в днях

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [9]:
data['children'].unique() # перечень уникальных значений

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [10]:
data = data[(data['children'] > -1) & (data['children'] < 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.

In [15]:
data['education'] = data['education'].str.lower() # нижний регистр

In [16]:
data.duplicated().sum() # количество дубликатов

71

In [17]:
data = data.drop_duplicates() # удаление дубликатов

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Будем использовать собственную функцию с именем `categorize_income()` и метод `apply()`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique() # перечень уникальных целей взятия кредита из столбца purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

Будем использовать собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


## Исследование данных

### Проверим есть ли зависимость между количеством детей и возвратом кредита в срок.

In [25]:
grouped_by_children = data.groupby('children')['debt'].agg(['count','sum','mean'])
grouped_by_children.columns = ['Количество людей', 'Количество людей с задолжностью', 'Доля людей с задолжностью']
grouped_by_children.sort_values(by='Доля людей с задолжностью')

,Количество людей,Количество людей с задолжностью,Доля людей с задолжностью
children,,,
5,9,0,0.000000
0,14091,1063,0.075438
3,330,27,0.081818
1,4808,444,0.092346
2,2052,194,0.094542
4,41,4,0.097561


**Вывод:** Зависимость между количеством детей и возвратом кредита в срок есть. Конверсия при количестве детей от 0 до 4 варьируется от 8 до 10 процентов. Если же детей 5, конверсия составляет 0 процентов. Данные результаты говорят о том, что люди у кого 5 детей составляют маленькую долю от всего количества. Можно добавить, что в данной категории данных не хватает, поэтому можно ее не учитывать. Также стоит отметить, что у категории людей без детей и категории людей у кого 3 ребенка в семье показывают лучшую конверсию, приблизительно равную 8 процентам, в отличии от других категорий.

### Проверим есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
grouped_by_family_status = data.groupby('family_status')['debt'].agg(['count','sum','mean'])
grouped_by_family_status.columns = ['Количество людей', 'Количество людей с задолжностью',
                                    'Доля людей с задолжностью']
grouped_by_family_status.sort_values('Доля людей с задолжностью')

,Количество людей,Количество людей с задолжностью,Доля людей с задолжностью
family_status,,,
вдовец / вдова,951,63,0.066246
в разводе,1189,84,0.070648
женат / замужем,12261,927,0.075606
гражданский брак,4134,385,0.093130
Не женат / не замужем,2796,273,0.097639


Поделим выбранные категории на две группы по формально оформленным отношениям.

In [27]:
def categorize_family_status(family_status):
    try:
        if family_status == 'женат / замужем':
            return 'оформленные отношения'
        return 'без отношений/не оформленные отношения'
    except:
        pass

In [28]:
data['family_status_category'] = data['family_status'].apply(categorize_family_status)

In [29]:
grouped_by_family_status_category = data.groupby('family_status_category')['debt'].agg(['count','sum','mean'])
grouped_by_family_status_category.columns = ['Количество людей', 'Количество людей с задолжностью',
                                             'Доля людей с задолжностью']
grouped_by_family_status_category

,Количество людей,Количество людей с задолжностью,Доля людей с задолжностью
family_status_category,,,
без отношений/не оформленные отношения,9070,805,0.088754
оформленные отношения,12261,927,0.075606


**Вывод:** Зависимость между семейным положением и возвратом кредита в срок также присутсвует. В категории вдовец/вдова наименьшая конверсия, это может быть связано с тем, что в данной категории меньше всего человек. Также видно, что люди в разводе и в категории женат/замужем имеют конверсию ниже, чем остальные категории. Дополнительно проанализировав данные, можно с уверенностью сказать, что заемщики в оформленном браке надежнее, чем без отношений.

### Проверим есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
grouped_by_total_income_category = data.groupby('total_income_category')['debt'].agg(['count','sum','mean'])
grouped_by_total_income_category.columns = ['Количество людей', 'Количество людей с задолжностью', 
                                            'Доля людей с задолжностью']
grouped_by_total_income_category.sort_values('Доля людей с задолжностью')

,Количество людей,Количество людей с задолжностью,Доля людей с задолжностью
total_income_category,,,
D,349,21,0.060172
B,5014,354,0.070602
A,25,2,0.080000
C,15921,1353,0.084982
E,22,2,0.090909


Поделим данные на другие группы, по уровню дохода. Первая группа с уровнем дохода до 100тыс., вторая до 200 тыс. и третья больше 200 тыс.

In [31]:
def categorize_income_new(income):
    try:
        if 0 <= income <= 100000:
            return 'A'
        if income <= 200000:
            return 'B'
        return 'C'
    except:
        pass

In [32]:
data['totla_income_category_new'] = data['total_income'].apply(categorize_income_new)

In [33]:
grouped_by_total_income_category_new = data.groupby('totla_income_category_new')['debt'].agg(['count',
                                                                                              'sum','mean'])
grouped_by_total_income_category_new.columns = ['Количество людей', 'Количество людей с задолжностью',
                                                'Доля людей с задолжностью']
grouped_by_total_income_category_new.sort_values('Доля людей с задолжностью')

,Количество людей,Количество людей с задолжностью,Доля людей с задолжностью
totla_income_category_new,,,
C,5039,356,0.070649
A,4444,354,0.079658
B,11848,1022,0.086259


**Вывод:** Зависимость между уровнем дохода и возвратом кредита в срок есть. Разделив на категории дохода, можно увидеть, что люди в категории D имеют наменьшую конверсию, что говорит нам о том, что в данной категории меньше всего задолжностей. Также стоит отметить категорию B, в которой меньше конверсия, чем в остальных категориях. В категории С намного больше людей, чем в остальных категориях. Исходя из дополнительного анализа данных, можно сделать вывод о том, что категория людей с доходом от 100 до 200 тыс. имеет самую высокую долю долгов (категория В). Проанализировав данные, можно сказать, что наиболее выгодной категорей для предоставления кредита является группа людей с доходом больше 200 тыс (категория С).

### Проверим как разные цели кредита влияют на его возврат в срок?

In [34]:
grouped_by_purpose = data.groupby('purpose_category')['debt'].agg(['count','sum','mean'])
grouped_by_purpose.columns = ['Количество людей', 'Количество людей с задолжностью', 'Доля людей с задолжностью']
grouped_by_purpose.sort_values('Доля людей с задолжностью')

,Количество людей,Количество людей с задолжностью,Доля людей с задолжностью
purpose_category,,,
операции с недвижимостью,10751,780,0.072551
проведение свадьбы,2313,183,0.079118
получение образования,3988,369,0.092528
операции с автомобилем,4279,400,0.093480


**Вывод:** По статистике, операции с недвижимостью показали лучшую конверсию, что может означать, что всего 7 процентов имеют задолжность по данной категории целей. Также стоит отметить, что проведение свадьбы имеет меньший процент, чем другие категории. Операции с автомобилем и получение образования, такие категории имеют больший процент задолжностей по кредитам.

### Приведем возможные причины появления пропусков в исходных данных.

Пропуски может допустить любой человек, случайный человеческий фактор, либо же могут быть технологические ошибки. Также могут быть случайные пропуски, но данные пропуски зависят от набора других данных в таблице. Существуют еще неслучайные пропуски, которые зависят есть набор данных или же его нет.

### Объяснение, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Медиана — это такое число в выборке, что ровно половина элементов больше него, а другая половина — меньше. Данные в выборке могут быть слишком приувеличены или приуменьшены, в таком случае среднее значение выборки искажает действительность. Медиана же, в свою очередь, даёт нам значение, которое находится по середине всей выборки, что является лучшим решением для количественных переменных. 

Допустим у нас в команде есть 5 человек с ростом в 180 см, пришел новый игрок с росто 215 см. В итоге среднее значение при 5 игроках 180 см, а при 6 - 185 см. По сути рост у 5 человек не изменился, но среднее значение больше, чем у всех 5 членов команды. Можно сделать вывод, что при слишком выделяющемся значении, лучше всего использовать медиану, т.к. она устойчива к сильно отклоняющимся значениям (устойчива к выбросам и аномалиям).

In [35]:
height = pd.Series([180, 180, 180, 180, 180]) #рост 5 игроков изначально
height_new = pd.Series([180, 180, 180, 180, 180, 215]) #рост 6 игроков 
print(height.mean(), height_new.mean())

180.0 185.83333333333334


## Общий вывод

**Вывод:** Решая данную задачу, можно сделать вывод о том, что любые данные необходимо проверять на пропуски, дубликаты и аномальные значения. Исследуя данные, нужно проанализировать все возможные варианты влияния разных категорий друг на друга. В заключении хотелось бы сказать, что проанализировав все данные, можно сказать, что выгодным заемщиком будут люди у которых нет детей, кто состоит в официальных отношениях, по зароботной плате в подкатегории С. Также лучше всего выбрать тех людей, чья цель кредита состоит в том, чтобы производить операции с недвижимостью.

In [36]:
data = data.loc[data['children'] == 0]
data = data.loc[(data['family_status_category'] == 'оформленные отношения')]
data = data.loc[(data['totla_income_category_new'] == 'C')]
data = data.loc[data['purpose_category'] == 'операции с недвижимостью']
data.reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,family_status_category,totla_income_category_new
0,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,оформленные отношения,C
1,0,1719.934226,35,среднее,1,женат / замужем,0,F,сотрудник,0,289202,недвижимость,B,операции с недвижимостью,оформленные отношения,C
2,0,529.191635,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,B,операции с недвижимостью,оформленные отношения,C
3,0,1682.083438,47,начальное,3,женат / замужем,0,F,сотрудник,0,275485,жилье,B,операции с недвижимостью,оформленные отношения,C
4,0,335.318630,36,высшее,0,женат / замужем,0,M,сотрудник,0,414404,недвижимость,B,операции с недвижимостью,оформленные отношения,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,0,9945.187489,48,среднее,1,женат / замужем,0,F,сотрудник,0,200223,операции с коммерческой недвижимостью,B,операции с недвижимостью,оформленные отношения,C
1001,0,2539.502794,52,высшее,0,женат / замужем,0,M,компаньон,0,527452,операции со своей недвижимостью,B,операции с недвижимостью,оформленные отношения,C
1002,0,1618.645283,54,среднее,1,женат / замужем,0,M,компаньон,0,295234,покупка жилой недвижимости,B,операции с недвижимостью,оформленные отношения,C
1003,0,1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986,покупка жилья,B,операции с недвижимостью,оформленные отношения,C


Также стоит отметить, что исключив категории людей с 4 детьми, без оформленных отношений, входящие в категорию доходов В, которые берут кредит для образования или выплаты за автомобиль будут улучшений в показателях возврата долга.